# Tutorial: Joining FLC catalogues

In this notebook we wil learn how to assemble standard mvs_targets_catalogue and unq_targets_catalogue to be fed to the pipeline to generate its own database.

## Setup and Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
from tqdm import tqdm
from astropy.io import fits
import datetime
import time
import numpy as np
from IPython.core.display import display, HTML
from straklip.utils.utils_tile import small_tiles

display(HTML("<style>.container { width:100% !important; }</style>"))

Here whe load the original catalog targets_drc.csv ina pandas dataframe. This catalog contain the following columns: target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr

In [ ]:
unq_target_df=pd.read_csv('/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/targets_drc.csv')
unq_target_df

In [ ]:
mvs_target_df=pd.read_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/targets_flc.csv')
mvs_target_df

In [ ]:
path2fits='/Users/gstrampelli/PycharmProjects/Giovanni/work/data/FFP/fits/'
for file in tqdm(glob(path2fits+'/*')):
    fitsname=file.split('/')[-1].split('.')[0]
    hdul=fits.open(path2fits+fitsname+'.fits')
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','filters']=hdul[0].header['FILTER']

    dateobs=hdul[0].header['DATE-OBS'].split('-')
    timeobs=hdul[0].header['TIME-OBS'].split(':')

    # df=pd.read_csv(file)
    date_time = datetime.datetime(int(dateobs[0]), int(dateobs[1]), int(dateobs[2]), int(timeobs[0]), int(timeobs[1]), int(timeobs[2]))
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','unittime']=time.mktime(date_time.timetuple())
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','fitsroot']=fitsname.split('_')[0]
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','filters']=hdul[0].header['FILTER']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','ccd']=hdul[1].header['CCDCHIP']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','visit']=str(fitsname[4:6])
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','exptime']=hdul[0].header['EXPTIME']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','pav3']=hdul[0].header['PA_V3']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','rota']=hdul[1].header['ORIENTAT']


In [ ]:
mvs_target_df

In [ ]:
mvs_target_df.loc[mvs_target_df.filters=='F814W']

In [ ]:
mvs_target_df.loc[mvs_target_df.filters=='F850LP']

In [ ]:
mvs_target_df.loc[~mvs_target_df.x.isna()]

In [ ]:
elno=0
# mvs_target_df['unq_ids'] = np.nan
for TARGET in mvs_target_df.TARGET.unique():
    mvs_target_df.loc[mvs_target_df.TARGET == TARGET, 'unq_ids'] = int(elno)
    elno+=1

In [ ]:
mvs_target_df = mvs_target_df.sort_values(['unq_ids','filters'])[['unq_ids', 'TARGET', 'file', 'x', 'y', 'dist', 'snr', 'filters','unittime','fitsroot','ccd','visit','exptime','pav3','rota']].reset_index(drop=True)

In [ ]:
mvs_target_df.loc[~mvs_target_df.x.isna()&(mvs_target_df.filters=='F814W')]

In [ ]:
mvs_target_df.loc[~mvs_target_df.x.isna()&(mvs_target_df.filters=='F850LP')]

In [ ]:
for target in mvs_target_df.TARGET.unique():
    if unq_target_df.loc[unq_target_df.target ==target].empty:
        index=mvs_target_df.loc[(mvs_target_df.TARGET==target)].index
        mvs_target_df.drop(index,inplace=True)
    else:
        unq_target_df.loc[unq_target_df.target ==target,'unq_ids'] = mvs_target_df.loc[mvs_target_df.TARGET == target, 'unq_ids'].values[0]
unq_target_df=unq_target_df.sort_values('unq_ids').reset_index(drop=True)
unq_target_df

In [ ]:
mvs_target_df

In [ ]:
pos=0
mvs_dataframe=pd.DataFrame(columns=['unq_ids','ext', 'visit','x_f814w','y_f814w','x_f850lp','y_f850lp','fitsroot_f814w','fitsroot_f850lp', 'exptime_f814w','exptime_f850lp', 'pav3_f814w', 'pav3_f850lp', 'rota_f814w', 'rota_f850lp','flag_f814w','flag_f850lp'])
mvs_dataframe['flag_f814w']='rejected'
mvs_dataframe['flag_f850lp']='rejected'

for id in tqdm(mvs_target_df.unq_ids.unique()):
    df_F814W = mvs_target_df.loc[(mvs_target_df.unq_ids==id)&(mvs_target_df.filters=='F814W')].sort_values(['unittime'])
    df_F850LP = mvs_target_df.loc[(mvs_target_df.unq_ids==id)&(mvs_target_df.filters=='F850LP')].sort_values(['unittime'])
    num =  df_F814W.unq_ids.count() if df_F814W.unq_ids.count() >= df_F850LP.unq_ids.count() else df_F850LP.unq_ids.count()
    for elno in range(num):
        mvs_dataframe.loc[pos,'unq_ids'] = id
        if not df_F814W.empty and elno <= df_F814W.unq_ids.count()-1:
            mvs_dataframe.loc[pos,'ext'] = 1 if df_F814W.ccd.unique() == 2 else 4
            mvs_dataframe.loc[pos,'visit'] = df_F814W.visit.unique()
            mvs_dataframe.loc[pos,['x_f814w','y_f814w','fitsroot_f814w','exptime_f814w','pav3_f814w','rota_f814w']] =df_F814W.iloc[elno][['x','y','fitsroot','exptime','pav3','rota']].values
        else:
            mvs_dataframe.loc[pos,'ext'] = 1 if df_F850LP.ccd.unique() == 2 else 4
            mvs_dataframe.loc[pos,'visit'] = df_F850LP.visit.unique()

        if not df_F850LP.empty and elno <= df_F850LP.unq_ids.count()-1:
            mvs_dataframe.loc[pos,['x_f850lp','y_f850lp','fitsroot_f850lp','exptime_f850lp','pav3_f850lp','rota_f850lp']] =df_F850LP.iloc[elno][['x','y','fitsroot','exptime','pav3','rota']].values
        pos+=1

mvs_dataframe

In [ ]:
mvs_dataframe = mvs_dataframe.reset_index().rename(columns={'index':'mvs_ids'})
mvs_dataframe['unq_ids']=mvs_dataframe.unq_ids.astype(int)
mvs_dataframe


In [ ]:
path2tiles='/Users/gstrampelli/PycharmProjects/Giovanni/work/out/FFP/'
bad_F814W=[124,166,192,193,194,197]
good_F814W=[4,5,6,7,14,30,43,48,50,88,90,98,116,120,122,124,126,129,141,146,148,164,165,167,168,172,174,177,184,185,186,187,188,189,190,191,192,193,194,195,196,198,199,200,201,202,203,208,209,210,211,212,216]

bad_F850LP=[166,184,185,189,190,192,197]
good_F850LP=[4,5,6,7,9,23,26,27,36,42,59,68,92,97,107,118,120,127,135,140,156,164,165,167,170,172,184,185,186,187,188,189,190,191,192,193,194,195,196,198,199,200,201,202,203,208,209,210,211,212,214,226]
dict={'bad_f814w':bad_F814W,'bad_f850lp':bad_F850LP,'good_f814w':good_F814W,'good_f850lp':good_F850LP}


mvs_dataframe = small_tiles(mvs_dataframe,path2fits, path2tiles, ['f814w','f850lp'], dict=dict,nrows=21, ncols=10)
mvs_dataframe

In [ ]:
mvs_dataframe.loc[mvs_dataframe['flag_f814w'].isna(),'flag_f814w'] = 'rejected'
mvs_dataframe.loc[mvs_dataframe['flag_f850lp'].isna(),'flag_f850lp'] = 'rejected'


In [ ]:
mvs_dataframe.to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_mvs_target_photometry.csv')

In [ ]:
pos=0
# unq_target_df = unq_target_df[['unq_ids','ra','dec','mag_psf','e_mag_psf']].sort_values(by='unq_ids').reset_index(drop=True)
unq_dataframe=pd.DataFrame(columns=['unq_ids','ra', 'dec','m_f814w','e_f814w','m_f850lp','e_f850lp','type','target'])

for id in tqdm(unq_target_df.unq_ids.unique()):
    unq_target_df.loc[(unq_target_df.unq_ids==id),'target']=unq_target_df.loc[unq_target_df.unq_ids==id].target.values[0]
    df_F814W = unq_target_df.loc[(unq_target_df.unq_ids==id)&(unq_target_df['filter']=='F814W')]
    df_F850LP = unq_target_df.loc[(unq_target_df.unq_ids==id)&(unq_target_df['filter']=='F850LP')]
    num =  df_F814W.unq_ids.count() if df_F814W.unq_ids.count() >= df_F850LP.unq_ids.count() else df_F850LP.unq_ids.count()
    for elno in range(num):
        unq_dataframe.loc[pos,'unq_ids'] = id
        if not df_F814W.empty:
            unq_dataframe.loc[pos,'ra'] = df_F814W.ra.unique()
            unq_dataframe.loc[pos,'dec'] = df_F814W.dec.unique()
        elif not df_F850LP.empty:
            unq_dataframe.loc[pos,'ra'] = df_F850LP.ra.unique()
            unq_dataframe.loc[pos,'dec'] = df_F850LP.dec.unique()

        unq_dataframe.loc[pos,'type'] = 1
        if np.all(mvs_dataframe.loc[mvs_dataframe.unq_ids==id,['flag_f814w','flag_f850lp']]=='bad_target'):
                unq_dataframe.loc[unq_dataframe.unq_ids==id,'type']=0
        if not df_F814W.empty and elno <= df_F814W.unq_ids.count()-1:
            unq_dataframe.loc[pos,['m_f814w','e_f814w']] =df_F814W.iloc[elno][['mag_aper','e_mag_aper']].values

        if not df_F850LP.empty and elno <= df_F850LP.unq_ids.count()-1:
            unq_dataframe.loc[pos,['m_f850lp','e_f850lp']] =df_F850LP.iloc[elno][['mag_aper','e_mag_aper']].values
        pos+=1

unq_dataframe=unq_dataframe.sort_values('unq_ids')
unq_dataframe['unq_ids']=unq_dataframe.unq_ids.astype(int)
unq_dataframe

In [ ]:
unq_dataframe.to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_unq_target_photometry.csv')

In [ ]:
unq_target_df[['unq_ids','target']].to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_avg2target_ids.csv')

In [ ]:
unq_target_df